In [4]:
# Detali vieno EKG įrašo klasifikacija su micro moduliu.
# Klasifikacija atliekama su micro moduliu, kuris klasifikuoja
# kiekvieną ECG pūpsnį atskirai, prieš tai apskaičiavus su kitu moduliu pūpsnio požymius.
# Tie požymiai gali būti iš anksto įrašyti į diską.

# Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
# Šis variantas pritaikytas npy formato zive įrašams, kuriems pakeistas, lyginant su 
# originaliais įrašais, failo vardas iš `file_name` į `SubjCode`, pridedant `userNr`
# prie `file_name`. 
#
# Skriptas zive EKG pūpsnių CNN VU klasifikatoriaus testavimui ir tikslumo įvertinimui, funkcijos 
# paimamos iš aplanko zive_cnn_fda_vu_v1.py, modelis iš model_cnn_fda_vu_v1, testuojami duomenys
# iš db_folder įrašų saugyklos, jame yra ir failas all_beats_attr. 

# Testavimui imami įrašai iš sąrašo SubjCodes, kuris arba paimamas if failo info_create.json,
# arba iš mokymo, validavimo, testavimo sarašų, pvz. train_subjcode_lst.csv. Visiems įrašams iš šių
# sąrašų egzistuoja informacija apie pūpsnius faile all_beats_attr.
 
# Skripte yra galimybė išvesti ekstrasistolių vietas įraše.
# Dirbant su daug įrašų reiktų užblokuoti: classification = []  # Užblokuota
 
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import sys, os, json
from pathlib import Path
# from icecream import ic
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support

from zive_util_vu import cm2df, show_confusion_matrix 
from zive_util_vu import create_dir, create_subdir, get_rev_dictionary
from zive_util_vu import runtime, split_SubjCode
from zive_util_vu import get_freq_unique_values

from zive_util_vu import get_beat_attributes
from zive_util_vu import get_userId, read_rec, get_filename 
from zive_util_vu import confusion_matrix_modified, zive_read_df_rpeaks

# from zive_cnn_fda_vu_v1 import classify_cnn_fda_vu_v1
# Pastaba: zive_read_file_1ch importuoju iš zive_cnn_fda_vu_v1, nors ji yra ir zive_util_vu.py
# tam, kad atskirti funkcijas, kurios importuojamos skripte zive analysis, nuo tų funkcijų,
# kurios reikalingos tik zive_accuracy_cnn_vu_v1 ir v2. 

print(tf.__version__)

print("Skriptas zive-arrh EKG segmentų apmokyto klasifikatoriaus tikslumo įvertinimui")
print('Modelis CNN VU su EKG sekos reikšmėmis, EKG formos požymiais, RR intervalais prieš ir po R dantelio')

import warnings
# warnings.filterwarnings("ignore")

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows, GUNDAS
    # Duomenu_aplankas = 'F:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows, HERKULIS
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE/VU'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas ir pūpsnių atributų failas
db_folder = 'DUOM_VU'

# Vietinės talpyklos aplankas ir pūpsnių atributų failas
all_beats_attr_fname = 'all_beats_attr_z.csv'

# Failai pūpsnių klasių formavimui
selected_beats = {'N':0, 'S':1, 'V':2}
all_beats =  {'N':0, 'S':1, 'V':2, 'U':3}  

# Diskretizavimo dažnis:
fs = 200

# /////////////////////////////////////////////////////////////////

#  Nuoroda į aplanką su MIT2ZIVE duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(db_path, 'records_npy')

# Nuoroda į modelio aplanką
# model_dir = Path(Duomenu_aplankas, 'DNN', 'best_models', 'all_ft')
model_dir = 'model_cnn_fda_vu_v0'

# Išvedame parametrus
print("\nBendras duomenų aplankas: ", Duomenu_aplankas)
print("Zive duomenų aplankas: ", db_folder)
print("Aplankas su originaliais EKG įrašais ir anotacijomis (.json) ", rec_dir)
print("Pūpsnių atributų failas:", all_beats_attr_fname)
print("Diskretizavimo dažnis: ", fs)
print('Klasifikavimo schema:', selected_beats)
print('Klasių skaičius:', len(selected_beats))
print("Modelio ir scaler parametrai nuskaitomas iš aplanko: ", model_dir)

print("\n")

2.8.2
Skriptas zive-arrh EKG segmentų apmokyto klasifikatoriaus tikslumo įvertinimui
Modelis CNN VU su EKG sekos reikšmėmis, EKG formos požymiais, RR intervalais prieš ir po R dantelio
OS in my system :  win32

Bendras duomenų aplankas:  D:\DI\Data\MIT&ZIVE\VU
Zive duomenų aplankas:  DUOM_VU
Aplankas su originaliais EKG įrašais ir anotacijomis (.json)  D:\DI\Data\MIT&ZIVE\VU\DUOM_VU\records_npy
Pūpsnių atributų failas: all_beats_attr_z.csv
Diskretizavimo dažnis:  200
Klasifikavimo schema: {'N': 0, 'S': 1, 'V': 2}
Klasių skaičius: 3
Modelio ir scaler parametrai nuskaitomas iš aplanko:  model_cnn_fda_vu_v0




In [5]:
# Modulis EKG signalo pūpsnių klasifikacijai su užduotu modeliu

import keras, pickle
from get_beat_features_fda_vu import get_beat_features_fda_vu_v1

def predict_cnn_fda_vu_v1_micro(signal, atr_sample, model_dir):
# ************************************* funkcijai ***************************************************************

#  Iėjimo parametrai:   signal, atr_sample, 
#                       modelio ir scaler_train parametrai
#                       
#  Išėjimo parametrai: pred_y, kai kurie neatpažinti : 'U':3

    # Suformuojame indeksų sąrašą. Formuodami sąrašą eliminuojame pirmą ir paskutinį indeksą

    all_features = ['RRl', 'RRr', 'RRl/RRr',
                'signal_mean', 'signal_std', 'P_val', 'Q_val', 'R_val', 'S_val', 'T_val',
                'P_pos', 'Q_pos', 'R_pos', 'S_pos', 'T_pos', 'QRS', 'PR', 'ST', 'QT', '0', '1', '2',
                '3', '4', '5', '6', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18',
                '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32',
                '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46',
                '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60',
                '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74',
                '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88',
                '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102',
                '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114',
                '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126',
                '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138',
                '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150',
                '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162',
                '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174',
                '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186',
                '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198',
                '199']
   
    print("pradėjome predict_cnn_fda_vu_v1")
    # nuskaitome modelio parametrus
    model_path = Path(model_dir, 'best_model_final_2.h5')
    model = keras.models.load_model(model_path)

    # Nuskaitome scaler objectą
    path_scaler = Path(model_dir, 'scaler.pkl')  
    scaler = pickle.load(open(path_scaler,'rb'))

    idx_lst = list(range(1, len(atr_sample)-1))
    # idx_lst = [1,2]

    # Formuojame iš pūpsnių požymių masyvą
    data_frame, omitted = get_beat_features_set_fda_vu_v1(signal, atr_sample, idx_lst)
    
    data_frame = data_frame.set_index('idx')
    data_frame.columns = data_frame.columns.astype(str)

    # print('\ndata_frame:')
    # print(data_frame.head(3))
    # print('\nomitted:')
    # print(omitted.head(3))

    # paruošiame požymių masyvą klasifikacijai
    data_frame_init = data_frame[all_features]
    data_array = scaler.transform(data_frame_init)
    test_x = data_array
    x_test = test_x.reshape((test_x.shape[0], test_x.shape[1], 1))

    # Pūpsnių klasių atpažinimas
    predictions = model.predict(x_test)
    predictions_y = np.argmax(predictions, axis=1)

# Sužymimi neatpažinti pūpsniai - klasė 3:'U'
    pred_y = np.zeros(len(atr_sample), dtype=int)
    pred_y[0] = 3
    pred_y[len(atr_sample)-1] = 3

    if (omitted.empty != True):
        idxs = list(omitted['idx'].astype('int'))
        for i in range(len(idxs)):
            pred_y[idxs[i]] = 3
            
# Sužymimi atpažinti pūpsniai 
    selected = data_frame.index.astype('int')
    pred_y[selected] = predictions_y
    
    # print(f"\natr_sample: {atr_sample[:40]}")

    return pred_y

# Apskaičiuojami užduotų EKG signalo pūpsnių (per idx_lst) požymiai ir iš jų
# suformuojamas požymių dataframe masyvas
def get_beat_features_set_fda_vu_v1(signal, atr_sample, idx_lst):

    beat_features_set = pd.DataFrame()
    omit_idx_set = pd.DataFrame()

    # print("\nGet beat features set from signal:")
    for idx in idx_lst:
        beat_features, omit_idx = get_beat_features_fda_vu_v1(signal, atr_sample, idx)
        # beat_features_set = beat_features_set.append(beat_features, ignore_index=True)
        if omit_idx.empty:
            beat_features_set = pd.concat([beat_features_set, beat_features])
        else:
            omit_idx_set = pd.concat([omit_idx_set, omit_idx])
    return beat_features_set, omit_idx_set

In [6]:
# Pagrindinis skriptas - ciklas per ECG įrašus

pd.set_option("display.max_rows", 6000, "display.max_columns",200)
pd.set_option('display.width', 1000)

import warnings
# warnings.filterwarnings("ignore")

print("\nAtliekama pūpsnių pacientų įrašuose klasifikacija")

# Pacientų įrašų sąrašas testavimui
file_path = 'testinis_sarasas.csv'
# SubjCodes = [10021, 10022, 10083, 10091] #Testavimui, iš mokymo imties
SubjCodes = [10000] #Testavimuas pacientų įrašų sąrašas

print("Klasifikuojamų įrašų sąrašas:", SubjCodes)
print(f"Sąrašas nuskaitytas iš: {file_path}")

# Kas kiek išvedamas apdorotų sekų skaičius
show_period = 100

# Klasių simbolinių vardų sąrašas ir klasių skaičius
class_names = list(selected_beats.keys()) 
n_classes = len(selected_beats)
# print(class_names)

# Nuskaitome pūpsnių atributų masyvą
file_path = Path(rec_dir, all_beats_attr_fname)
all_beats_attr = pd.read_csv(file_path, index_col=0, dtype = {'userNr': int, 'recordingNr': int,
                                                        'sample': int, 'symbol': str, 'label': int})
all_beat_indices = all_beats_attr.index

index_start = 0
# Sukūriame masyvą, į kurį sudėsime visų įrašų pūpsnių anotuotus ir automatiškai surastus klasių numerius
validation_set_stats = pd.DataFrame(columns=['idx', 'test_label', 'pred_label', 'SubjCode'])

start_time = time.time()
# Ciklas per pacientų įrašus
for SubjCode in SubjCodes:
    
    # Nuskaitome EKG įrašą (npy formatu)
    sign_raw = read_rec(rec_dir, SubjCode)
    signal_length = sign_raw.shape[0]
    signal = sign_raw

    # Surandame ir išvedame įrašo atributus
    print("\nZive įrašas:")
    file_name = get_filename(rec_dir, SubjCode)
    userNr, recNr = split_SubjCode(SubjCode)
    print(f"SubjCode: {SubjCode} userNr: {userNr:>2} file_name: {file_name:>2} signal_length: {signal_length}")
    
    filepath = Path(rec_dir, str(SubjCode) + '.json')
    with open(filepath,'r', encoding='UTF-8', errors = 'ignore') as f:
        data = json.loads(f.read())
    userId = data['userId']
    recordingId = data['recordingId']
    print(f"userId: {userId}  recordingId: {recordingId}")

    # Filtruojame signalą
    # signal = signal_filter(signal=sign_raw, sampling_rate=200, lowcut=0.2, method="butterworth", order=5)

    # Nuskaitome paciento anotacijas ir jų indeksus
    df_rpeaks = zive_read_df_rpeaks(rec_dir, str(SubjCode))
    atr_sample = df_rpeaks['sampleIndex'].to_numpy()
    atr_symbol = df_rpeaks['annotationValue'].to_numpy()
  

    # SUFORMUOJAME EKG ĮRAŠUI TESTINĮ ir PRISKIRTŲ KLASIŲ NUMERIŲ MASYVUS

    # Jei pasitaiko symbol 'U' arba 'F', pūpsniui suteikiame klasę 3, kurią vėliau apvalysime  
    test_labels = np.array([all_beats[symbol] for symbol in atr_symbol])

    (unique, counts) = np.unique(atr_symbol, return_counts=True)
    total = counts.sum()
    print("test_symbols: ", unique, counts, total)

    (unique, counts) = np.unique(test_labels, return_counts=True)
    total = counts.sum()
    print("test_labels: ", unique, counts, total)

    print(f"\natr_sample: {atr_sample[:total]}")
    print(f"\natr_symbol: {atr_symbol[:total]}")

    pred_labels = predict_cnn_fda_vu_v1_micro(signal, atr_sample, model_dir)
    # pred_labels turi būti tokio pat ilgio, kaip ir test_labels, praleisti (šiuo atveju pirmas
    # ir paskutinis pūpsnys), o taip pat pakliuvęs į ommited sritį, pažymimi klase 3
    if (len(test_labels) != len(pred_labels)):
        raise Exception(f"Klaida! SubjCode: {SubjCode}. Nesutampa test_labels ir pred_labels ilgiai")     

    (unique, counts) = np.unique(pred_labels, return_counts=True)
    total = counts.sum()
    print("pred_labels: ",unique, counts, total)

    symbol_dict = dict(zip(all_beats.values(), all_beats.keys()))
    # print(symbol_dict)

    # Surandame vietas su ekstrasistolemis ir išvedame jų sąrašą vizualiniam įvertinimui. 
    classification=[]
    for i, i_sample in enumerate(atr_sample):
        if ((pred_labels[i] != 0) or test_labels[i] != 0):
            classification.append({'sample':i_sample, 'annot':symbol_dict[test_labels[i]], 'pred':symbol_dict[pred_labels[i]]})

    # Vietų sąrašas išvedamas
    # Dirbant su daug įrašų sąrašo išvedimą reikia užblokuoti !!!!!!!!!!!!!!!!!!!!!!!!!!!
    # classification = []  # Užblokuota
    if (classification):
        tot = 0
        tot_U = 0
        for row in classification:
            print(f"sample: {row['sample']:>7}   annot_label: {row['annot']:>2}   pred_label: {row['pred']:>2}")
            if (row['pred'] != 'U'):
                tot += 1
            else:
                tot_U +=1     

    print(f"rpeaks total: {len(atr_sample)}")
    print(f"excluded rpeaks ('U'): {tot_U}")
    print(f"classified rpeaks: {len(atr_sample) - tot_U}")
    print(f"automaticRpeakAnnotations: {tot}")

    # SUFORMUOJAME FREIMĄ validation_set_stats SU PŪPSNIŲ KLASIŲ ANOTUOTAIS IR AUTOMATIŠKAI 
    # SURASTAIS KLASIŲ NUMERIAIS, IŠMETANT KLASES SU NUMERIU = 3

    # Surandame pradinį SubjCode įrašo indeksą faile all_beats_attr
    selected_ind = all_beat_indices[(all_beats_attr['userNr']==userNr) & (all_beats_attr['recordingNr']==recNr)]
    # print(f"SubjCode: {SubjCode}  first elem: {selected_ind[0]} last elem: {selected_ind[-1]}  tot: {len(selected_ind)}")
    index_start = selected_ind[0]
    # print('\nSubjCode:',SubjCode, 'index_start:', index_start)   

    #  Praleisdami indeksą, jei masyvuose test_labels ir pred_labels yra reikšmė == 3,
    # suformuojame klasifikuotinų pūpsnių indeksų sąrašą
    for idx in range(len(atr_sample)):
        flag = (test_labels[idx] == 3) or (pred_labels[idx] == 3)
        if (flag == False):
            validation_stats = pd.DataFrame([{'idx':index_start+idx,'test_label':test_labels[idx],
                                                'pred_label':pred_labels[idx], 'SubjCode': SubjCode}])
            validation_set_stats = pd.concat([validation_set_stats, validation_stats])

    # Suformuojame klasių numerių msyvus confusion matricai skaičiuoti, surandama confusion matrica
    test_y = np.array(validation_set_stats[validation_set_stats['SubjCode']==SubjCode]['test_label']).astype('int') 
    # print(all_beats_attr.info())
    pred_y = np.array(validation_set_stats[validation_set_stats['SubjCode']==SubjCode]['pred_label']).astype('int')
   
    # Atsikračius pūpsnių su klase = 3 ir suformavus masyvus, pred_y turi būti tokio pat ilgio, kaip ir test_y
    if (len(test_y) != len(pred_y)):
        raise Exception(f"Klaida! SubjCode: {SubjCode}. Nesutampa test_y ir pred_y ilgiai")     

    # Skaičiuojame ir išvedame klasifikavimo lentelę
    confusion = confusion_matrix(test_y, pred_y)
    # print(confusion)
    pd.set_option('display.precision',3)
    show_confusion_matrix(confusion, class_names)

# print('\n')
    prec,rec,fsc,sup = precision_recall_fscore_support(test_y, pred_y, labels=[0, 1, 2], zero_division=0)

    str1 =f"N:{int(sup[0]):>5} S:{(int(sup[1])):3} V:{int(sup[2]):3}" 
    str2 = f"  Nprec:{prec[0]:>5.2f} Nrec:{rec[0]:5.2f} Nfsc:{fsc[0]:5.2f}"
    str3 = f"  Sprec:{prec[1]:>5.2f} Srec:{rec[1]:5.2f} Sfsc:{fsc[1]:5.2f}"
    str4 = f"  Vprec:{prec[2]:>5.2f} Vrec:{rec[2]:5.2f} Vfsc:{fsc[2]:5.2f}"
    print(str1+str2+str3+str4)

    # classification = classify_cnn_fda_vu_v1(signal, atr_sample, model_dir, prediction_labels = ['N', 'S', 'V', 'U'])
    # print(classification)
    # df_classification = pd.DataFrame(classification)
    # print(df_classification.value_counts())

    # print(len(validation_set_stats))
    # print(len(test_y))
    # print(len(pred_y))

end_time = time.time()
print('\n')
runtime(end_time-start_time)

# Sukūriame anotuotų ir automatiškai priskirtų klasių visų įrašų pūpsniams sąrašus 
validate_ind_lst = list(validation_set_stats['idx'])
y_validate = np.array(validation_set_stats['test_label']).astype('int')
y_predicted = np.array(validation_set_stats['pred_label']).astype('int')



Atliekama pūpsnių pacientų įrašuose klasifikacija
Klasifikuojamų įrašų sąrašas: [10000]
Sąrašas nuskaitytas iš: testinis_sarasas.csv

Zive įrašas:
SubjCode: 10000 userNr: 1000 file_name: 1630673.825 signal_length: 127999
userId: 6034c808d6c2740008035ede  recordingId: 6133101f3d08d40385cdc8ce
test_symbols:  ['N' 'S'] [762   4] 766
test_labels:  [0 1] [762   4] 766

atr_sample: [   166    333    506    681    853   1028   1205   1371   1530   1688
   1841   1998   2156   2313   2467   2622   2775   2931   3088   3244
   3399   3554   3707   3860   4016   4173   4331   4493   4662   4832
   5014   5203   5385   5571   5754   5929   6108   6286   6458   6632
   6810   6987   7170   7357   7534   7715   7899   8079   8264   8454
   8636   8823   9011   9188   9370   9553   9729   9911  10094  10276
  10462  10645  10819  10993  11168  11337  11504  11672  11842  12008
  12177  12349  12523  12701  12879  13053  13227  13404  13579  13754
  13931  14105  14275  14451  14631  14805  14985  1

In [7]:
# Nesutvarkyta !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Sukuriami užduoto ilgio sekų vaizdai ir įrašomi į disko atitinkamus klasėms subfolderius

from zive_util_vu import get_rec_Id, split_seq_file_name, get_symbol_list, get_SubjCode, create_SubjCode
from zive_util_vu import read_show_seq_ext_zive, read_seq, get_seq_start_end, read_rec_attrib


def read_show_seq_ext_zive_npy(rec_dir, all_beats_attr, idx, win_ls, win_rs, win_ls_ext, win_rs_ext):
# Išpjauna užduoto ilgio seką iš mit2zive įrašo ir sukuria jos vaizdą su anotacijomis

# rec_dir - paciento EKG įrašų aplankas
# recordingId - paciento EKG įrašo Id - int
# i_sample - R dantelio, kurio atžvilgiu formuojama seka, indeksas viso EKG įrašo reikšmių masyve - int
# win_ls - klasifikuojamo EKG segmento plotis iki R pūpsnio (iš kairės) 
# win_rs - klasifikuojamo EKG segmento plotis nuo R pūpsnio (iš dešinės)
# win_ls_ext - vaizduojamo EKG segmento plotis iki R pūpsnio (iš kairės) 
# win_rs_ext - vaizduojamo EKG segmento plotis už R pūpsnio (iš dešinės) 

    ax = plt.gca()

    sequence, sample, label = read_seq(rec_dir, all_beats_attr, idx, win_ls_ext, win_rs_ext)
    if (sample == None):
        print("klaida!")
        return None
    
    seq_start = sample - win_ls_ext
    seq_end = sample + win_ls_ext

    SubjCode = get_SubjCode(idx, all_beats_attr)

    # Nuskaitome paciento anotacijas ir jų indeksus
    atr_sample, atr_symbol = read_rec_attrib(rec_dir, SubjCode)

    # # suformuojame anotacijų žymes
    beat_symbols,beat_locs = get_symbol_list(atr_symbol,atr_sample, seq_start, seq_end)

    # deltax ir deltay simbolių pozicijų koregavimui
    min = np.amin(sequence)
    max = np.amax(sequence)
    deltay = (max - min)/20
    deltax = len(sequence)/100

    # suformuojame vaizdą
    x = np.arange(0, len(sequence), 1)
    ax.plot(x, sequence, color="#6c3376", linewidth=2)
    left_mark = win_ls_ext - win_ls
    right_mark = win_ls_ext + win_rs
    ax.axvline(x = left_mark, color = 'b', linestyle = 'dotted')
    ax.axvline(x = right_mark, color = 'b', linestyle = 'dotted')
    for i in range(len(beat_locs)):
        ax.annotate(beat_symbols[i],(beat_locs[i]-deltax,sequence[beat_locs[i]]+deltay))
    ax.set_ylim([min, max+2*deltay])
    
    return(ax)

def get_seq_attributes(all_beats_attr, idx):
    # 'userNr', 'recordingNr', 'sample', 'symbol', 'label', 'RRl', 'RRr'
    row = all_beats_attr.loc[idx]
    return list(row) 

# ////////////////////// Užduodami parametrai /////////////////////

# Bendras aplankas vaizdams
images_folder = 'CNN'

# užduodame, kiek reikšmių vaizduosime prieš R dantelį ir po
wl_side_ext = 360
wr_side_ext = 360

# Užduodame, kiek sekų vaizdų iš kiekvienos klasės įrašysime į diską
img_max = 10

# /////////////////////////////////////////////////////////////////

print("\nBendras aplankas vaizdams:", images_folder)
print("\nMax sekų vaizdų skaičius iš kiekvienos klasės:", img_max)

if (img_max == 0):
    sys.exit()

# sukuriame bendrą aplanką vaizdams
images_dir = Path(sets_path, 'saved_images', images_folder)
create_dir(images_dir)

# klasių simbolinių vardų sąrašas
class_names = selected_beats.keys()

# sukuriame aplankus sekų vaizdams klasėse
create_subdir(images_dir, class_names)

# Sukuriame vaizdų klasėse skaitiklį
n_classes = len(class_names)
skait = np.zeros((n_classes, n_classes), dtype=int)
# print(skait)

# sukuriame selected_beats reversiją
rev_dict = get_rev_dictionary(selected_beats)

y_test = df_seq_errors['labels'].to_numpy(dtype=int)
y_pred = df_seq_errors['preds'].to_numpy(dtype=int)


# *********************** derinimui *******************************
# acc = accuracy_score(y_test, y_pred)
# print(f"\nAccuracy: {acc:.2f}\n")
# cnf_matrix = confusion_matrix_modified(y_test, y_pred, n_classes)
# show_confusion_matrix(cnf_matrix, class_names)
# ******************************************************************

# Ciklas per sekas

icycle = 0 
leng = len(y_pred)

for idx in validate_ind_lst:
    if (icycle >= leng):
        continue

# *********************** derinimui ******************************   
    # row = get_seq_attributes(all_beats_attr, idx)
    # print(idx, row)
# ****************************************************************

    # anotuotos klasės (klasių nr ir simboliniai pažymėjimai)
    label = y_test[icycle]
    label_symb = rev_dict[label]

    # print(f" {idx} label {label} symb {label_symb}")

    # klasifikatoriaus priskirtos klasės (klasių nr ir simboliniai pažymėjimai)
    pred = y_pred[icycle]
    pred_symb = rev_dict[pred]
    icycle +=1

    # print(f" {idx} pred {pred} symb {pred_symb}")

    # patikriname, ar neviršytas skaitiklis, jei viršytas, peršokame
    if (skait[label,pred] >= img_max):
        continue
    else:
        skait[label,pred] += 1

    SubjCode = get_SubjCode(idx, all_beats_attr)
    seq_name = str(SubjCode) + '_' + str(idx)
    
     # 'Išpjauname' užduoto ilgio sekas ir sukuriame jų vaizdus
    fig = plt.figure(facecolor=(1, 1, 1), figsize=(18,3))
    # print("rec_dir =", rec_dir)
    ax = read_show_seq_ext_zive_npy(rec_dir, all_beats_attr, idx, wl_side, wr_side, wl_side_ext, wr_side_ext) 
    if (ax == None):
        print(f'Sekai {idx} negali suformuoti išplėstinio vaizdo')
        plt.close()
        continue

    # suformuosime koreguotą failo vardą
    file_name = seq_name + '_' + pred_symb + ".png" 

    # suformuosime kelią į atitinkamą sub-aplanką
    image_subdir = Path(images_dir, label_symb)
    if (os.path.exists(image_subdir) == False):
        print('Klaida! ', image_subdir,' neegzistuoja')
    file_path = Path(image_subdir, file_name)
    # print('file_name: ',file_name, 'file_path: ', file_path)   

    # Įrašome į atitinkamą anotacijai sub-aplanką 
    ax.set_title(file_name)
    plt.savefig(file_path, bbox_inches='tight', pad_inches = 0.2)
    plt.close()

    # if (icycle >= len(y_pred)):
        # break

# ciklo per validate_ind_lst pabaiga

print("\n")
df = cm2df(skait, class_names)
print(df)

print("\nPabaiga.........")


ImportError: cannot import name 'split_seq_file_name' from 'zive_util_vu' (d:\DI\ZIVEO_2022_RUDUO\VU_APRIL_tesinys\zive_util_vu.py)

: 